#### Wesleyan University ASTR 221

## Tutorial 6: Numerical Integration ("Quadrature")

There are two types of circumstances when one might want to integrate something numerically: when the underlying functional form is unknown, or when it is so complicated that evaluating the integral analytically is difficult or impossible.  In either case, we will want to use samples of the function at discrete points to estimate the total area under the curve.  

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120  
from math import pi

Below I define the function ```f(x)``` that we want to integrate.  In this case, I've made it a 4th-degree polynomial, but you can define it however you want!  The second function, ```int_f()``` gives the (analytically-known) definite integral of that function.

In [ ]:
# Function
def f(x):
    return 10*x - 5*x**2 + 7*x**3 - x**4

# Definite integral of function above from xmin to xmax
def int_f(xmin, xmax):
    return -xmax**5/5 + 7*xmax**4/4 - 5*xmax**3/3 + 5*xmax**2 - (-xmin**5/5 + 7*xmin**4/4 - 5*xmin**3/3 + 5*xmin**2)

# Construct array of equally-spaced points
xmin = 0.
xmax = 6
npts = 10

# Sample points
x = np.linspace(xmin, xmax, npts)
y = f(x)

# Make a very high-resolution x-grid to show the underlying curve
x_hr = np.linspace(xmin, xmax, 200)
y_hr = f(x_hr)

# Plot the curve, sample points, and area under the curve
fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', lw=5, color='deepskyblue')
plt.fill_between(x_hr, y_hr, color='blue', alpha=0.5)
plt.plot(x, y, 'o', ms=5, color='red')
ax.set_xlabel('x')
ax.set_ylabel('y')

We'll be exploring methods for calculating the area in dark blue: the integral
$$\int_{x_0}^{x_{n-1}} f(x)~dx.$$
Because we know the underlying functional form, we know the exact value of this integral - but how close can we get to this value with numerical approximations, using only our discrete sample points?

The most basic **Newton-Cotes method** - that is, a method which uses the sample points to define shapes that describe the area under the curve - is the *rectangle rule*.  Visually, it looks something like this:

In [ ]:
fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', lw=5, color='deepskyblue')
plt.plot(x, y, 'o', ms=5, color='red')
plt.bar(x[:-1], y[:-1], width=(x[1]-x[0]), edgecolor='blue', align='edge', color='blue', alpha=0.5)
ax.set_xlabel('x')
ax.set_ylabel('y')

The area of each rectangle is base x height, where the base is $\Delta x$ and the height is $f(x_i)$.  The full area under the curve is therefore approximated by
$$\int_{x_0}^{x_{n-1}} f(x)~dx \approx \sum_{i=0}^{n-2} f(x_i) \Delta x.$$
Note that in this approximation, we do not use the last sample point.

Let's write a function to perform this sum for us:

In [ ]:
# Estimates the definite integral of y over the range given by x
# Assumes that x and y are numpy arrays
def int_rectangle(x,y):
    deltax = x[1:] - x[:-1]
    return np.sum(y[:-1]*deltax)

How close are we?  By visual inspection, we must be undercounting, but we can check directly since we know the true value of the integral:

In [ ]:
print(int_rectangle(x,y)/int_f(xmin, xmax))

so we have an error of about 7%.  This will improve if we increase the number of points:

In [ ]:
npts = 20
dx = (xmax - xmin)/(npts-1)

# Sample points
x = np.linspace(xmin, xmax, npts)
y = f(x)

fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', lw=5, color='deepskyblue')
plt.plot(x, y, 'o', ms=5, color='red')
plt.bar(x[:-1], y[:-1], width=(x[1]-x[0]), edgecolor='blue', align='edge', color='blue', alpha=0.5)
ax.set_xlabel('x')
ax.set_ylabel('y')

print(int_rectangle(x,y)/int_f(xmin,xmax))

Our estimate now looks much closer, and indeed our error has gone down to about 3%.

Next, let's consider the **trapezoid rule**.  Rather than drawing a rectangle between $x_i$ and $x_{i+1}$, we draw a straight line between the points and use that to define a *trapezoid*, like so:

In [ ]:
npts = 7
dx = (xmax - xmin)/(npts-1)

# Sample points
x = np.linspace(xmin, xmax, npts)
y = f(x)

fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', lw=3, color='deepskyblue')
plt.plot(x, y, 'o', ms=5, color='red')

# Draw a trapezoid between each pair of points
for i in range(npts-1):
    plt.fill_between([x[i], x[i+1]], [y[i], y[i+1]], edgecolor='blue', color='blue', alpha=0.5)

ax.set_xlabel('x')
ax.set_ylabel('y')

The area of each trapezoid is
$$\int_{x_i}^{x_{i+1}} f(x)~dx \approx \frac{f(x_i) + f(x_{i+1})}{2} \Delta x,$$
so the sum over all trapezoids is
$$\int_{x_0}^{x_{n-1}} f(x) dx \approx (f(x_0)/2 + f(x_1) + f(x_2) + ... + f(x_{n-2}) + f(x_{n-1})/2)~\Delta x $$
where every value of $f(x_i)$ is multiplied by $\Delta x$ except for the endpoints, which are multiplied by $\Delta x/2$.  This is equivalent to
$$\left(\sum_{i=0}^{n-1} f(x_i) \Delta x \right) - (f(x_0) + f(x_{i-1})) \frac{\Delta x}{2},$$
which is how the integral is calculated in the function below:

In [ ]:
# Estimates the definite integral of y over the range given by x
# Assumes that x and y are numpy arrays
# Assumes that the points are evenly spaced
def int_trapezoid(x,y):
    deltax = x[1] - x[0]
    return (np.sum(y) - y[0]/2 - y[-1]/2)*deltax

# Calculate the integral using the trapezoid rule and compare it to the true value
print(int_trapezoid(x,y)/int_f(xmin, xmax))

Here we can see that with only 7 points, the trapezoid rule is already out-performing the rectangle rule at 20 points!

Our final Newton-Cotes method is to use sets of three points (i.e., every two intervals or *panels*) to define **parabolas**:
$$f(x_i < x < x_{i+2}) \approx A(x-x_i)^2 + B(x-x_i) + C.$$
We have discussed this before in the context of interpolation; the solution for the coefficients $A$, $B$, and $C$ is referred to as **Simpson's rule**.  Below I show the parabolas that are defined by 7 sample points on our function:

In [ ]:
npts = 7
dx = (xmax - xmin)/(npts-1)

# Sample points
x = np.linspace(xmin, xmax, npts)
y = f(x)

fig, ax = plt.subplots()
plt.plot(x_hr,y_hr, '-', lw=3, color='deepskyblue')
plt.plot(x, y, 'o', ms=3, color='red')
for i in np.arange(0,npts-1,2):
    A = (y[i] - 2*y[i+1] + y[i+2])/(2*(x[1]-x[0])**2)
    B = (-3*y[i] + 4*y[i+1] - y[i+2])/(2*(x[1]-x[0]))
    C = y[i]
    inrange = (x_hr >= x[i]) & (x_hr <= x[i+2])
    xplt = x_hr[inrange]
    plt.fill_between(xplt, A*(xplt-x[i])**2 + B*(xplt-x[i]) + C, edgecolor='blue', color='blue', alpha=0.5, lw=1.5)
ax.set_xlabel('x')
ax.set_ylabel('y')

You can see already by eye that this is a pretty stellar approximation!  The Simpson's rule parabolas integrate to
$$ \int_{x_i}^{x_{i+2}} f(x)~dx \approx \frac{\Delta x}{3}(f(x_i) + 4f(x_{i+1}) + f(x_{i+2}),$$
so the sum over all panels gives
$$\int_{x_0}^{x_{n-1}} f(x) dx \approx \frac{\Delta x}{3}(f(x_0) + 4f(x_1) + 2f(x_2) + 4f(x_3) + ... + 2f(x_{n-3}) + 4f(x_{n-2}) + f(x_{n-1})) $$
for odd $n$.  This method cannot be applied for even $n$ (because each parabola uses two intervals so there would be one left over), but one can tack on a trapezoid at one end if need be.  (If so, this is often where most of the error will come from!)  The function below implements this method:

In [ ]:
# Estimates the definite integral of y over the range given by x
# Assumes that x and y are numpy arrays
# Assumes that the points are evenly spaced
# Works best if there are an odd number of points, but will handle an even number
def int_parabola(x,y):
    
    # If number of panels is odd (number of points is even), need to remove last point for now
    if len(x) % 2 == 0:
        y_parab = y[:-1]
    else:
        y_parab = y
    
    # Integrate using Simpson's rule (parabolic interpolation)
    deltax = x[1] - x[0]
    parab = np.sum(4*y_parab[1::2]) + np.sum(2*y_parab[0::2]) - y_parab[0] - y_parab[-1]
    parab = parab*deltax/3.
    
    # Tack on a trapezoid for the last panel if number of panels is odd
    if len(x) % 2 == 0:
        parab += (y[-1] + y[-2])*deltax/2.
        
    return parab

# Evaluate the integral using this function and compare to the true value
print(int_parabola(x,y)/int_f(xmin, xmax))

Wow!  With only three parabolas (7 points), the error is only 0.2\%!

Finally, let's take a look at how the error in all three methods changes as a function of the number of sample points (i.e., resolution).  

In [ ]:
nmin = 3  # Should be odd
nmax = 21

# Evaluate the integral using each method for every odd number of points between nmin and nmax
estimate_rectangle = [int_rectangle(np.linspace(xmin, xmax, npts), f(np.linspace(xmin, xmax, npts))) for npts in np.arange(nmin, nmax+1,2)]
estimate_trapezoid = [int_trapezoid(np.linspace(xmin, xmax, npts), f(np.linspace(xmin, xmax, npts))) for npts in np.arange(nmin, nmax+1,2)]
estimate_parabola = [int_parabola(np.linspace(xmin, xmax, npts), f(np.linspace(xmin, xmax, npts))) for npts in np.arange(nmin, nmax+1,2)]

In [ ]:
fig, ax = plt.subplots()
alln = np.arange(nmin, nmax+1,2)
trueval = int_f(xmin, xmax)

# Plot the error relative to the true value as a function of number of points
plt.plot(alln, 100*(trueval-np.array(estimate_rectangle))/trueval, '-', color='red', lw=3, label='Rectangle')
plt.plot(alln, 100*(trueval-np.array(estimate_trapezoid))/trueval, '-', color='gold', lw=3, label='Trapezoid')
plt.plot(alln, 100*(trueval-np.array(estimate_parabola))/trueval, '-', color='blue', lw=3, label='Parabola')

ax.set_xlabel(r'$N_{pts}$')
ax.set_ylabel('% Error')
ax.set_ylim(1e-3,50)
ax.set_yscale('log')
ax.legend()

There are many other, more sophisticated methods out there for integrating functions numerically.  These usually apply to the scenario where you know the functional form, but it is not analytically tractable - in that case, you have freedom to choose the evaluation (sample) points, and there are clever ways to choose them that can increase the accuracy and speed of the calculation.  (See: Gaussian quadrature)  There are also higher-order Newton-Cotes methods, which use a larger number of points to define the shapes.

As a **rule of thumb**, however: the simple trapezoid rule is pretty robust against weird behavior in your functions, compatible with linear interpolation between points if you want to integrate to intermediate values, and usually accurate enough to get you where you want to go!